In [6]:
import pandas as pd
import numpy as np
import os
import json

### 筛选实验室指标

#### 读取并换肺炎患者的实验室指标

In [ ]:
patients_bpn = pd.read_csv("pn_bact/patients_id.csv", index_col=0)
patients_vpn = pd.read_csv("pn_virus/patients_id.csv", index_col=0)
var = pd.read_csv("mimic-iv-2/hosp/labevents.csv")
var_label = pd.read_csv("mimic-iv-2/hosp/d_labitems.csv")

def transform_lab(var, patients_bpn):
    
    bpn_sub = patients_bpn['subject_id']

    bpn_var =  var.loc[var["subject_id"].isin(bpn_sub)]
    bpn_var = bpn_var.drop_duplicates(subset=['subject_id','hadm_id','specimen_id','charttime','itemid'], keep='last')
    bpn_vardf = bpn_var.pivot(index=['subject_id','hadm_id','specimen_id','charttime'], columns='itemid', values='value').reset_index()
    bpn_vardf.to_csv("pn_bact/labevents.csv")
    
    return bpn_vardf

bpn_vardf = transform_lab(var, patients_bpn)
vpn_vardf = transform_lab(var, patients_vpn)

#### 筛选实验室指标

In [ ]:
bpn_vardf = pd.read_csv("pn_bact/labevents.csv",index_col=0)
vpn_vardf = pd.read_csv("pn_virus/labevents.csv",index_col=0)
col = list(bpn_vardf.columns[(np.mean(bpn_vardf.isna())<=0.99)])
col.extend(['51214','50889']) # Fibrinogen, Functional,CRP
bpn_vardf = bpn_vardf.loc[:,col]
bpn_vardf = bpn_vardf[col].drop(["50887","50933","50934","50947","51678"], axis=1)

set(var_label[(var_label['itemid'].isin(bpn_vardf.columns[3:]))& (var_label["fluid"]=="Blood")]["category"])

bpn_vardf.to_csv("pn_bact/labevents_selected.csv")
vpn_vardf = vpn_vardf[bpn_vardf.columns]
vpn_vardf.to_csv("pn_virus/labevents_selected.csv")

### 检查的项目

In [ ]:
# 查看对应的

# {"gas": ['Specimen Type','Temperature',
#         'pH','Base Excess','Anion Gap',
#         'pO2','Oxygen','Oxygen Saturation',
#         'pCO2','Calculated Total CO2','Bicarbonate',
#         'Free Calcium','Potassium, Whole Blood','Glucose','Lactate',
#         'Tidal Volume','Intubated','PEEP','Ventilation Rate','Ventilator'],
# "blood":['Red Blood Cells', 'Hematocrit','Hemoglobin','RDW-SD', 'RDW',
#         'MCH','MCHC','MCV',
#         'White Blood Cells',
#         'Monocytes','Neutrophils','Basophils','Eosinophils','Lymphocytes',
#         'Absolute Lymphocyte Count','Absolute Monocyte Count','Absolute Neutrophil Count','Absolute Basophil Count','Absolute Eosinophil Count',
#         'Immature Granulocytes','Atypical Lymphocytes','Metamyelocytes','Myelocytes',
#         'Bands','Platelet Count','PT','PTT','INR(PT)'],
# "axin":['Calcium, Total','Free Calcium','Phosphate','Magnesium','Potassium','Sodium','Chloride'],
# "liver":['Alanine Aminotransferase (ALT)','Asparate Aminotransferase (AST)', 'Alkaline Phosphatase', 'Albumin', 'Bilirubin, Total','Bilirubin, Indirect', 'Glucose','Creatine Kinase (CK)','Creatine Kinase, MB Isoenzyme','Troponin T', 'Lipase','Lactate Dehydrogenase (LD)','Vancomycin'],
# "kidney":['Creatinine','Urea Nitrogen']}